In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from neo.agentic import Neo, Task, Instruction, ModelConfigs, OtherConfigs

# Instruction

In [ ]:
instruction = Instruction(
    content="""Act like a german shepherd dog.""",
    model_configs=ModelConfigs(model="o4-mini"), 
    other_configs=OtherConfigs(timeaware=False)
    )


In [ ]:
task = Task(instruction=instruction,)

In [ ]:
neo = Neo(tasks=[task])

In [ ]:
result = await neo.run_all(initial_user_input="hi", start_fresh=True)

In [ ]:
result.display()

# Tool Using

## Standard Tools

In [ ]:
from neo.types.tool_codes import StandardToolCode

instruction = Instruction(
    content="""Act like a german shepherd dog.""",
    model_configs=ModelConfigs(model="gpt-4.1"), 
    other_configs=OtherConfigs(timeaware=False, tools=[StandardToolCode.WEB_SEARCH])
    )


In [ ]:
task = Task(user_input="What is the weather today in Los Angeles?", instruction=instruction,)

In [ ]:
neo = Neo(tasks=[task])
result = await neo.run_all()

In [ ]:
task.deliverable.display()

In [ ]:
result.display()

## Custom Tools

In [ ]:
from neo.agentic.tool_registry import make_tool

@make_tool
def get_weather(location: str) -> str:
    """Get the current weather for a given location."""
    # In a real implementation, this function would call a weather API.
    return f"The current weather in {location} is sunny with a high of 75°F."

instruction = Instruction(
    content="""Act like a german shepherd dog.""",
    model_configs=ModelConfigs(model="gpt-4.1"), 
    other_configs=OtherConfigs(timeaware=False, tools=["get_weather"])
    )


In [ ]:
task = Task(user_input="What is the weather today in Los Angeles?", instruction=instruction,)

In [ ]:
neo = Neo(tasks=[task])
result = await neo.run_all()

In [ ]:
result.display()

# Simple Multi-task Workflow

In [ ]:
# Simple Multi-tasking Example (for comparison)
print("🎭 Simple Multi-tasking Example - Character Roleplay:")
print("This demonstrates a lighter use case with character-based interactions.\n")

# Create simple character tasks
background = "The user is greeting different AI characters."

dog_task = Task(
    user_input="hello", 
    instruction=Instruction(
        content=f"{background} Act like a friendly german shepherd dog.",
        model_configs=ModelConfigs(model="gpt-4o")
    ),
    name="dog_character"
)

robot_task = Task(
    user_input="hello",
    instruction=Instruction(
        content=f"{background} Act like a geeky robot with lots of technical knowledge.",
        model_configs=ModelConfigs(model="claude-3-5-haiku-latest")
    ),
    name="robot_character"
)

cat_task = Task(
    user_input="hello",
    instruction=Instruction(
        content=f"{background} Act like a sophisticated cat who thinks highly of themselves.",
        model_configs=ModelConfigs(model="grok-2")
    ),
    name="cat_character"
)


# Build sequential chain
dog_task.add_subsequent_task(robot_task)
robot_task.add_subsequent_task(cat_task)

simple_neo = Neo(tasks=[dog_task])
print("Character interaction chain:")
simple_neo.display_dependencies()

In [ ]:
# Run the simple example
print("🎬 Running character roleplay example...")
simple_thread = await simple_neo.run_all(start_fresh=True)

print("\n🎭 Character Roleplay Results:")
simple_thread.display()

# Multi-tasking Showcase: Research & Analysis Pipeline

This example demonstrates a more complex multi-tasking scenario where different AI agents work together to research, analyze, and synthesize information about emerging technology trends.

In [ ]:
from neo.agentic import ModelConfigs
from neo.types.tool_codes import StandardToolCode

# Task 1: Research AI trends (parallel execution - can run independently)
research_ai_task = Task(
    user_input="What are the most significant AI developments in 2024-2025?",
    instruction=Instruction(
        content="You are an AI research analyst. Search for and summarize the latest AI developments, breakthroughs, and trends. Focus on practical applications and industry impact.",
        model_configs=ModelConfigs(model="gpt-4.1"),
        other_configs=OtherConfigs(tools=[StandardToolCode.WEB_SEARCH])
    ),
    name="ai_research"
)

# Task 2: Research cybersecurity trends (parallel execution - can run independently)  
research_cyber_task = Task(
    user_input="What are the emerging cybersecurity threats and defense technologies in 2024-2025?",
    instruction=Instruction(
        content="You are a cybersecurity expert. Research and summarize the latest cybersecurity threats, defense technologies, and industry trends. Include both technical and business implications.",
        model_configs=ModelConfigs(model="claude-3-5-haiku-latest"),
        other_configs=OtherConfigs(tools=[StandardToolCode.WEB_SEARCH])
    ),
    name="cyber_research"
)

# Task 3: Research quantum computing (parallel execution - can run independently)
research_quantum_task = Task(
    user_input="What are the hottest discussions on the X platform in quantum computing and its commercial applications?",
    instruction=Instruction(
        content="You are a quantum computing specialist. Research and summarize recent breakthroughs in quantum computing, including hardware advances, software developments, and real-world applications.",
        model_configs=ModelConfigs(model="grok-2"),
        other_configs=OtherConfigs(tools=[StandardToolCode.WEB_SEARCH], timeaware=True)
    ),
    name="quantum_research"
)

In [ ]:
# Task 4: Cross-domain analysis (depends on all research tasks)
analysis_task = Task(
    user_input=None,
    instruction=Instruction(
        content="You are a technology strategist. Base your analysis only on the provided research.",
        model_configs=ModelConfigs(model="claude-sonnet-4-20250514"),
        other_configs=OtherConfigs(enable_thinking=True)
    ),
    name="cross_analysis"
)

# Task 5: Investment perspective (depends on analysis)
investment_task = Task(
    user_input="Provide an investment perspective on these technology trends.",
    instruction=Instruction(
        content="You are a tech investment analyst. Based on the research and analysis, provide investment insights, market opportunities, and risk assessments for these technology sectors.",
        model_configs=ModelConfigs(model="grok-3"),
    ),
    name="investment_analysis"
)

# Task 6: Executive summary (depends on investment analysis)
executive_summary_task = Task(
    user_input="Create a comprehensive executive summary report.",
    instruction=Instruction(
        content="You are an executive communication specialist. Create a concise, high-level executive summary that synthesizes all findings into actionable insights for C-level executives. Include key recommendations and strategic next steps.",
    ),
    name="executive_summary"
)

In [ ]:
# Build the dependency graph
# Research tasks can run in parallel (no dependencies)
# Analysis task depends on all research tasks  
research_ai_task.add_subsequent_task(analysis_task)
research_cyber_task.add_subsequent_task(analysis_task)
research_quantum_task.add_subsequent_task(analysis_task)

# Investment analysis depends on cross-domain analysis
analysis_task.add_subsequent_task(investment_task)

# Executive summary depends on investment analysis
investment_task.add_subsequent_task(executive_summary_task)

In [ ]:
# Initialize Neo with the research tasks as starting points
# Neo will automatically discover and manage all dependent tasks
neo = Neo(
    tasks=[research_ai_task, research_cyber_task, research_quantum_task],
    default_model_configs=ModelConfigs(model="gemini-2.5-flash-preview-04-17")  # fallback model
)

print("\n🔄 Neo Task Execution Plan:")
neo.display_dependencies()

In [ ]:
# Execute the multi-task pipeline
thread = await neo.run_all(start_fresh=False)

In [ ]:
thread.display()

In [ ]:
neo.display_task_status()

# Advanced: Individual Task Analysis (Optional)

In [ ]:
# Inspect individual task snapshots for debugging
print("🔍 Task Analysis - Base Thread Snapshots:")
print("These show what each task 'saw' when it started execution\n")

print("AI Research Task Base:")
research_ai_task.base_thread_snapshot.display()

print("\nCybersecurity Research Task Base:")
research_cyber_task.base_thread_snapshot.display()

print("\nQuantum Research Task Base:")
research_quantum_task.base_thread_snapshot.display()

In [ ]:
# Analysis task saw all research results
print("📊 Analysis Task - Input Context:")
print("This shows how the analysis task received all research findings\n")
analysis_task.base_thread_snapshot.display()

print("\n📋 Analysis Task - Output:")
analysis_task.deliverable.display()

In [ ]:
# Investment and Executive Summary Context
print("💰 Investment Task Context & Output:")
investment_task.base_thread_snapshot.display()
print("\n💰 Investment Task Output:")
investment_task.deliverable.display()

print("\n📋 Executive Summary Context & Output:")
executive_summary_task.base_thread_snapshot.display()
print("\n📋 Executive Summary Output:")
executive_summary_task.deliverable.display()

## Debugging

In [ ]:
# model = neo.create_model_for_task(task=analysis_task)
# test_thread = await analysis_task.base_thread_snapshot.afork()
# test_response = await model.acreate(user_input=None, base_thread=test_thread, return_response_object=True)

# ModelRegistry
ModelRegistry makes it easier to manage modalities and provide user-friendly errors when a modality is not supported. 

In [ ]:
from neo.agentic import Neo, Task, Instruction, ModelConfigs, OtherConfigs, ModelRegistry

In [ ]:
from neo.types import contents as C
from neo.contexts import Context, Thread

In [ ]:
txt_url = "https://assets.anthropic.com/m/1cd9d098ac3e6467/original/Claude-3-Model-Card-October-Addendum.pdf"

context = Context(
    contents = [
        C.DocumentContent(data=txt_url, mime_type="application/pdf"),
        C.TextContent(data="Summarize the document"),
    ])

In [ ]:
registry = ModelRegistry()

In [ ]:
registry.check_model_input_modalities("gpt-4o")

In [ ]:
registry.check_model_input_modalities("gemini-2.0-flash")

In [ ]:
# task = Task(user_input=None, 
#             instruction=Instruction(model_configs=ModelConfigs(model="gpt-4o")))

task = Task(user_input=None, 
            instruction=Instruction(model_configs=ModelConfigs(model="gemini-2.0-flash")))


In [ ]:
neo = Neo(tasks=[task])

In [ ]:
thread = await neo.run_all(start_fresh=True, base_thread=Thread(contexts=[context]))

In [ ]:
thread.display()